<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />Peter Br&ouml;cker</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">&Uuml;bungsblatt 5</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2016</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml)

**Abgabe**: Montag, 23. Mai, 2016 vor der Vorlesung

**Name**: <font color="red">Bitte geben Sie ihren Namen an</font>

**Matrikelnummer**: <font color="red">Bitte geben Sie ihre Matrikelnummer an</font>

<table style="border-style: none; width: 100%"><tr style="border-style: none;">
<td style="border-style: none; width: 1%; font-size: 20px; font-weight: bold">Monde auf der Überholspur</td>
<td style="border-style: none; width: 1%; text-align: right">(5 Punkte)</td></tr></table>

Die Monde **Janus** und **Epimetheus** umkreisen den Planeten Saturn *koorbital*, d.h. auf nahezu der gleichen Bahn.
Bemerkenswerterweise ist dabei der Bahnunterschied zwischen dem inneren und dem äußeren der beiden Monde deutlich
kleiner als deren Durchmesser. Alle vier Jahre begegnen sich die beiden Monde, da der innere Mond ein wenig schneller unterwegs ist als der Äußere. Wieso es dabei nicht zu einer Kollision kommt, wollen wir in dieser Aufgabe untersuchen.

Dazu wollen wir das **3-Körper-System** aus Saturn, Janus und  Epimetheus simulieren. Für die Simulation bewegen wir uns ins Bezugssystem des Saturn. Zwischen zwei Körpern wirkt dann eine Gravitationskraft gemäß

\begin{equation}
\mathbf{F}_{12}=-G\frac{m_1 m_2}{|\mathbf{r}_1-\mathbf{r}_2|^3}\left(\mathbf{r}_1-\mathbf{r}_2\right),
\end{equation}

wobei wir die Gravitationskonstante $G = 1$ setzen (wodurch sich die Größen der Parameter natürlich von den realistischen unterscheiden). Berechnen Sie die Bahnen der Monde mittels des **Verlet**-Verfahrens und plotten Sie die Bahnkurven der Monde. Verwenden Sie folgende Startwerte:

\begin{align*}
m_1 &= 1 \\
m_2 &= 4 \\
m_{\text{Saturn}} &= 4\times10^4 \\
\mathbf{r}_1 &= (-155, 0) \quad \frac{d\mathbf{r}_1}{dt} = (0, -16.1) \\
\mathbf{r}_2 &= (150, 0), \quad \frac{d\mathbf{r}_2}{dt} = (0, 16.3)
\end{align*}